# APRENDIZADO POR REFORÇO
### **TAREFA 1**: Algoritmo *Temporal Difference (TD)* para o problema do robô de reciclagem
### **ALUNOS**: 
- ### Alex Júnio Maia de Oliveira (*Matrícula: 241708028*)
- ### João Pedro Jerônimo de Oliveira (*Matrícula: 241708024*) 
- ### Thalis Ambrosim Falqueto (*Matrícula: 241708048*)
---

[Introdução](#introdução)

[Metodologia](#metodologia)

[Resultados](#resultados)

[Conclusão](#conclusão)

## Introdução

O problema do robô móvel de reciclagem é um exemplo clássico da aprendizagem por reforço. Neste projeto, implementamos o problema utilizando o algoritmo *Temporal Difference Learning*, baseado na ideia de que um agente pode aprender a partir de suas próprias ações e das recompensas ou penalidades que recebe em função dessas ações, ou seja, o agente aprende a partir da diferença entre as recompensas esperadas e as recompensas reais obtidas (erro temporal).

Abaixo está uma apresentação breve do problema do robô móvel, motivador desse projeto:

"Um robô móvel tem a tarefa de coletar latas de refrigerante vazias em um ambiente de escritório. Ele possui sensores para detectar latas, além de um braço e uma garra capazes de pegá-las e colocá-las em um compartimento interno; o robô funciona com uma bateria recarregável. O sistema de controle do robô possui componentes para interpretar informações sensoriais, para navegação e para controlar o braço e a garra. Decisões de alto nível sobre como procurar por latas são tomadas por um agente de reinforcement learning, com base no nível atual de carga da bateria.

Para simplificar o exemplo, assumimos que apenas dois níveis de carga podem ser distinguidos, formando um pequeno conjunto de estados $S=\{high, low\}$. Em cada estado, o agente pode decidir entre:

- *search*: procurar ativamente por uma lata durante um certo período de tempo,
- *wait*: permanecer parado e esperar que alguém lhe traga uma lata, ou
- *recharge*: retornar à sua base para recarregar a bateria.

Quando o nível de energia está high, recarregar seria sempre uma decisão tola, portanto não incluímos essa opção no conjunto de ações para esse estado. Assim, os conjuntos de ações são: $A(high)=\{search, wait\}$ e $A(low)=\{search, wait, recharge\}$.

As *rewards* são geralmente zero, mas tornam-se positivas quando o robô consegue uma lata vazia, ou grandes e negativas caso a bateria se esgote completamente. A melhor forma de encontrar latas é fazendo *search*, mas isso consome a bateria do robô, enquanto *wait* não consome. Sempre que o robô estiver em search, existe a possibilidade de a bateria se esgotar. Nesse caso, o robô deve desligar-se e esperar para ser resgatado (produzindo uma baixa *reward*).

Se o nível de energia estiver *high*, então um período de *search* pode ser concluído sem risco de esgotar a bateria. Um período de *search* iniciado com nível *high* mantém o nível *high* com probabilidade $α$ e o reduz para *low* com probabilidade $1-\alpha$. Por outro lado, um período de *search* iniciado quando o nível de energia está *low* mantém-no *low* com probabilidade $𝛽$ e esgota a bateria com probabilidade $1 - 𝛽$. Nesse último caso, o robô deve ser resgatado, e a bateria é então recarregada para *high*.

Cada lata coletada pelo robô conta como uma unidade de *reward*, enquanto uma penalidade de $−3$ ocorre sempre que o robô precisa ser resgatado. Sejam $r_{search}$ e $r_{wait}$, com $r_{search}$ > $r_{wait}$, respectivamente o número esperado de latas que o robô coleta (e, portanto, a *reward* esperada) enquanto faz *search* e enquanto faz *wait*. Por fim, suponha que nenhuma lata possa ser coletada durante o retorno para *recharge*, nem em um passo no qual a bateria se esgota.

Esse sistema é, portanto, um *finite MDP*, e podemos escrever as probabilidades de transição e as *expected rewards*, com dinâmicas indicadas na tabela à esquerda."

<p align="center">
  <img src="figures/image.png" alt="Diagrama" width="500"/>
</p>


<p align="center">
  *Figura 1. Diagrama dos agentes, das recompensas e do ambiente do problema*
</p>

## Metodologia

Com o objetivo de otimizar o tempo de desenvolvimento e facilitar a construção do robô, dividimos o projeto em três partes principais:

- **Ambiente** - Responsável por processar as recompensas de acordo com os estados e com as interações do robô, além de calibrar as probabilidades $α$ e $β$ e a taxa de aprendizado do agente (desenvolvido por João Pedro).
- **Agente ou Robô** - Responsável por administrar as probabilidades de cada estado e escolher e aprender a política para obter a maior recompensa possível, de acordo com uma taxa de aprendizado (desenvolvido por Thalis Ambrosim).
- **Gráficos** - Responsável por produzir os gráficos das recompensas acumuladas juntamente com a curva acumulada média, do mapa de calor das probabilidades da política ótima do robô e das curvas médias acumuladas de acordo com taxas de aprendizado diferentes (desenvolvido por Alex Júnio).

Além das vertentes supracitadas, criamos uma interface dinâmica e mutável para acompanhar o treinamento do robô de reciclagem com diferentes parâmetros (probabilidade $α$ e $β$, taxa de aprendizado e número de épocas). Ao decorrer da estruturação da organização do projeto, decidimos modularizar o código nas seguintes pastas:

```bash
|--config  # Pasta que contém algumas configurações de ambiente, para facilitar o código
|--figures # Pasta que guarda os plots gerados pelo código
|--report  # Pasta que guarda o relatório do projeto
|    |--figures  # Pasta que armazena as figuras utilizadas pelo relatório
|    |-- report_recycle_robot.ipynb
|--rewards  # Pasta que contém o arquivo rewards.txt, que guarda a recompensa total ao final de um período de E épocas
|    |-- rewards.txt
|--src  # Pasta que contém o código fonte do problema do robô de reciclagem
|    |-- ambient.py
|    |-- plot_funcs.py
|    |-- robot.py
|--main.py  # Arquivo que executa a aprendizagem do robô, armazena os plots e gera o arquivo rewards.txt
```

Em adição, dados os valores $α$ (probabilidade do robô começar e terminar uma pesquisa com a bateria alta)e $β$ (probabilidade do robô começar e terminar uma pesquisa com a bateria baixa), achamos interessandte implementarmos uma maneira das probabilidades decairem conforme o tempo da pesquisa passa (seguindo uma distribuição Exponencial), simulando o tempo de vida bateria do robô.

Por fim, dadas diferentes taxas de aprendizagem, calculamos a curva média das recompensas acumuladas para observar qual taxa gera a melhor evolução média do robô de acordo com a quantidade de épocas e as probabilidades $\alpha$ e $\beta$.


## Resultados

Ao fazer várias rodadas de teste alterando os parâmetros do robô, observamos que:
- o valor esperado das recompensas recompensas gira em torno de $0.8$ (em que a curva média da política ótima fica mais perto desse valor) e converge quando o número de épocas tende ao infinito.
- o valor acumulado das recompensas da política ótima gira em torno de $800$ ao final das épocas.
- na maior parte das vezes (independentemente dos valores de $α$, $β$, do número de épocas e da taxa de aprendizado), quando o robô está com a bateria alta

## Conclusão